# PatMatch Use With a Genome and Protein Sequences + Python

Here I'll demonstrate use of the stand-alone, command line based PatMatch on a genome sequence and protein sequences. 
Hopefully, you'll see the the script you used previously to move to Python will work for such data cases as well.

## Preparing to use PatMatch software on genomic and protein sequence data



*Genome file*

As far as the genome, normally I'd recommnend obtaining the *Saccharomyces cerevisiae* budding yeast from [Illumina’s iGenomes site](https://support.illumina.com/sequencing/sequencing_software/igenome.html), and in fact, I have written out the commands necessary on the standard command line [here](http://fenglabwkshopmay2015.readthedocs.io/en/latest/genome%20of%20interest%20data%20acquisition/). [Illumina’s iGenomes site](https://support.illumina.com/sequencing/sequencing_software/igenome.html) is a collection of reference sequences and annotation files for commonly analyzed organisms.  However, it seems FTP-based and FTP sources won't work via Binder, and instead the genome file has been included already as `Sc_genome.fa`. (This has the advantage that it spares users from waiting a few seconds for the download to process.)

Alternatively, the following code pasted in a Jupyter notebook cell will work in Jupyter environments run via Binder to get the genome sequnce from [here](https://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/). However, the genome file, `S288C_reference_sequence_R64-2-1_20150113.fsa`, from this source doesn't seem to contain nicely labeled chromosomes on the FASTA description lines and instead has cryptic alphanumerical code identifiers.

```
!curl -O https://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz
!tar -xzf S288C_reference_genome_Current_Release.tgz
!cp S288C_reference_genome_R64-2-1_20150113/S288C_reference_sequence_R64-2-1_20150113.fsa .
!rm -rf S288C_reference_genome_R64-2-1_20150113
!rm S288C_reference_genome_Current_Release.tgz
```

Given those cryptic labels, we'll stick with the included FASTA genome file.

*Protein sequences*

Following advice from [here](https://www.biostars.org/p/106065/#106110) is it is easy to select several and get quite a few at once from [here](http://www.uniprot.org/uniprot/?query=%28taxonomy%3A9606%29+AND+reviewed%3Ayes) because it formats a link when choose 'download' and designate 'uncompressed'. That link obtained can be used to make a `curl` command to get the text.  
Running the next cell will get a file of several human protein sequences.

In [ ]:
!curl "https://www.uniprot.org/uniprot/?sort=&desc=&compress=no&query=id:P62258%20OR%20id:P61981%20OR%20id:P31947%20OR%20id:P63104%20OR%20id:P30443%20OR%20id:Q16537%20OR%20id:Q30167%20OR%20id:Q95IE3%20OR%20id:Q5Y7A7%20OR%20id:P46952%20OR%20id:P31937%20OR%20id:P30542%20OR%20id:Q9NS82%20OR%20id:Q9NRG9%20OR%20id:Q2M2I8%20OR%20id:O43741%20OR%20id:P54619%20OR%20id:Q9UNQ0%20OR%20id:Q9H172%20OR%20id:Q9H222%20OR%20id:Q8N2K0%20OR%20id:Q96I13%20OR%20id:Q8IZP0%20OR%20id:P49748%20OR%20d:Q5HYA8%20OR%20id:Q9UHC1%20OR%20id:Q9UHV7%20OR%20id:P52732%20OR%20id:Q15058%20OR%20id:Q9P2E2%20OR%20id:Q9P2G9%20OR%20id:Q86Z14%20OR%20id:Q8NB78%20OR%20id:Q9Y4C1&format=fasta" > human_examples.txt

If that download fails, paste `!cp human_examples_bckup.txt human_examples.txt` into a cell and run it so that the later steps will still work.

## Running PatMatch on Genomic Sequences and sending results into a Python dataframe

Prepare the FASTA file per usual.

In [ ]:
!perl ../patmatch_1.2/unjustify_fasta.pl Sc_genome.fa

Now run `PatMatch`.

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" Sc_genome.fa.prepared > entire_genome.out

You'll need the script you used prior to convert the results to a dataframe. To insure that it is here, you can run this next cell.

In [ ]:
!curl -O https://raw.githubusercontent.com/fomightez/sequencework/master/patmatch-utilities/patmatch_results_to_df.py

With that script, the results for the entire genome can be made into a dataframe.

In [ ]:
%run patmatch_results_to_df.py entire_genome.out --pattern DDWDWTAWAAGTARTADDDD -name promoter

In [ ]:
import pandas as pd
genome_df = pd.read_pickle("patmatch_pickled_df.pkl")
genome_df

Having the data in a dataframe it is easy to tally the hit numbers by chromosome.

In [ ]:
genome_df.sort_values('hit_number', ascending=False, inplace=True)
hit_nums_by_chr_df = genome_df.groupby('FASTA_id').head(1)
hit_nums_by_chr_df

## Running PatMatch on Protein Sequences and sending results into a Python dataframe

Prepare the FASTA file obtained per usual.

In [ ]:
!perl ../patmatch_1.2/unjustify_fasta.pl human_examples.txt

Now run `PatMatch`.

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -p "STEP" human_examples.txt.prepared 1 s > human_protein.out

Bring the results into Python dataframe. Since we brought the script into where this notebook is running earlier, we can run the Python script already. We just have to add the `--protein_results` flag to process the results as protein data because the script defaults to handling as if from nucleic acids and the dataframe will include irrelevant items otherwise.

In [ ]:
%run patmatch_results_to_df.py human_protein.out --pattern STEP -name our_motif --protein_results

In [ ]:
import pandas as pd
protein_df = pd.read_pickle("patmatch_pickled_df.pkl")
protein_df

------

Next
----

Return to the [index](../index.ipynb) or continue on with the next notebook to see demonstration of a way to skip having the script save and then immediately bring that file back in to Python when you are actively perfoming pattern matching.